In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
data = pd.read_csv('./Customer_Subscription_And_Transaction_Details.csv') # 데이터 로드
data = data.dropna() # 결측값 처리
data

,cust_id,transaction_type,transaction_date,subscription_type,subscription_price,customer_gender,age_group,customer_country,referral_type
0,1,initial,2020-09-01,BASIC,33,Male,35-44,Denmark,Google Ads
1,2,initial,2022-03-01,BASIC,53,Female,18-24,Denmark,facebook
2,3,initial,2020-10-01,MAX,99,Female,25-34,Norway,facebook
3,3,REDUCTION,2022-02-01,BASIC,53,Female,25-34,Norway,facebook
4,4,initial,2022-05-01,PRO,85,Male,18-24,Sweden,Display
...,...,...,...,...,...,...,...,...,...
18101,10364,UPGRADE,2022-09-01,MAX,119,Female,45-54,Norway,facebook
18102,10365,initial,2021-05-01,PRO,75,Male,55-65,Norway,Google Ads
18103,10365,REDUCTION,2021-07-01,BASIC,43,Male,55-65,Norway,Google Ads
18104,10366,initial,2021-01-01,BASIC,43,Male,18-24,Denmark,facebook


In [4]:
data = pd.get_dummies(data,columns=['subscription_type', 'customer_country', 'transaction_type', 'customer_gender', 'referral_type', 'age_group']) # 원핫인코딩
data['transaction_date'] = pd.to_datetime(data['transaction_date']) #날짜 타임형식 변환

In [5]:
tmp_count = data[data['transaction_type_CHURN']]['cust_id'].value_counts() #CHURN 중복 찾기
tmp_count = tmp_count[tmp_count.values > 1] #CHURN 카운트 1이면 중복 아님

In [6]:
tmp_count #CHURN이 2와 3인 고객들

cust_id
7483    3
4479    3
9127    3
3946    3
6913    3
       ..
2859    2
4873    2
5067    2
9839    2
7348    2
Name: count, Length: 135, dtype: int64

In [7]:
(tmp_count[(tmp_count.values==3)].index) #CHURN이 3인 tmp_count의 인덱스가 cust_id

Index([7483, 4479, 9127, 3946, 6913, 9765, 8093, 3865], dtype='int64', name='cust_id')

In [11]:
data[data['cust_id'].isin(tmp_count[(tmp_count.values==3)].index)][data['transaction_type_CHURN']==True] #이 리스트에서 각 cust_id들의 마지막 행만 살리고 1,2번째는 원래 리스트에서 빠져야 하는 중복 CHURN 데이터

,cust_id,transaction_date,subscription_price,subscription_type_BASIC,subscription_type_MAX,subscription_type_PRO,customer_country_Denmark,customer_country_Finland,customer_country_Norway,customer_country_Sweden,...,referral_type_Organic Search,referral_type_Paid Search,referral_type_TV,referral_type_Unknown,referral_type_facebook,age_group_18-24,age_group_25-34,age_group_35-44,age_group_45-54,age_group_55-65
6738,3865,2021-12-01,43,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
6739,3865,2022-03-01,119,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
6740,3865,2022-04-01,119,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
6885,3946,2021-06-01,43,True,False,False,False,False,False,True,...,False,False,False,False,True,False,True,False,False,False
6886,3946,2021-10-01,75,False,False,True,False,False,False,True,...,False,False,False,False,True,False,True,False,False,False
6887,3946,2021-11-01,43,True,False,False,False,False,False,True,...,False,False,False,False,True,False,True,False,False,False
7835,4479,2021-09-01,75,False,False,True,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
7836,4479,2021-11-01,43,True,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
7837,4479,2022-03-01,53,True,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
12080,6913,2021-07-01,75,False,False,True,False,True,False,False,...,True,False,False,False,False,False,False,True,False,False


In [12]:
tmp_del = data[data['cust_id'].isin(tmp_count[(tmp_count.values==3)].index)][data['transaction_type_CHURN']==True] #중복된 것들 중 CHURN이 3개인 cust_id 만 모아서 그 중 CHURN을 제외한 type은 제외한 데이터프레임
result = tmp_del.groupby('cust_id',as_index=False,group_keys=False).apply(lambda x: x.iloc[:2]) #tmp_del에 있는 CHURN 중에서 최종 탈퇴 CHURN 행만 빼고 나머지를 데이터프레임으로 만드는 코드(인덱스 유지를 위해 False로)

In [14]:
result

,cust_id,transaction_date,subscription_price,subscription_type_BASIC,subscription_type_MAX,subscription_type_PRO,customer_country_Denmark,customer_country_Finland,customer_country_Norway,customer_country_Sweden,...,referral_type_Organic Search,referral_type_Paid Search,referral_type_TV,referral_type_Unknown,referral_type_facebook,age_group_18-24,age_group_25-34,age_group_35-44,age_group_45-54,age_group_55-65
6738,3865,2021-12-01,43,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
6739,3865,2022-03-01,119,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
6885,3946,2021-06-01,43,True,False,False,False,False,False,True,...,False,False,False,False,True,False,True,False,False,False
6886,3946,2021-10-01,75,False,False,True,False,False,False,True,...,False,False,False,False,True,False,True,False,False,False
7835,4479,2021-09-01,75,False,False,True,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
7836,4479,2021-11-01,43,True,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
12080,6913,2021-07-01,75,False,False,True,False,True,False,False,...,True,False,False,False,False,False,False,True,False,False
12081,6913,2022-02-01,119,False,True,False,False,True,False,False,...,True,False,False,False,False,False,False,True,False,False
13094,7483,2022-03-01,53,True,False,False,True,False,False,False,...,False,False,True,False,False,False,True,False,False,False
13095,7483,2022-08-01,85,False,False,True,True,False,False,False,...,False,False,True,False,False,False,True,False,False,False


In [15]:
data.drop(result.index,inplace=True) #원본 data에서 result의 인덱스가 그대로 인덱스를 가지고 있으므로 인덱스를 기준으로 사이에 껴있는 CHURN 행 삭제

In [ ]:
#중복이 2인 애들 다시 반복

In [16]:
tmp_del = data[data['cust_id'].isin(tmp_count[(tmp_count.values==2)].index)][data['transaction_type_CHURN']==True] #중복된 것들 중 CHURN이 2개인 cust_id 만 모아서 그 중 CHURN을 제외한 type은 제외한 데이터프레임
result = tmp_del.groupby('cust_id',as_index=False,group_keys=False).apply(lambda x: x.iloc[:1]) #tmp_del에 있는 CHURN 중에서 최종 탈퇴 CHURN 행만 빼고 나머지를 데이터프레임으로 만드는 코드(인덱스 유지를 위해 False로)

In [17]:
data.drop(result.index,inplace=True) #원본 data에서 result의 인덱스가 그대로 인덱스를 가지고 있으므로 인덱스를 기준으로 사이에 껴있는 CHURN 행 삭제

In [ ]:
#data.to_csv('drop_dataset.csv')

In [18]:
data

,cust_id,transaction_date,subscription_price,subscription_type_BASIC,subscription_type_MAX,subscription_type_PRO,customer_country_Denmark,customer_country_Finland,customer_country_Norway,customer_country_Sweden,...,referral_type_Organic Search,referral_type_Paid Search,referral_type_TV,referral_type_Unknown,referral_type_facebook,age_group_18-24,age_group_25-34,age_group_35-44,age_group_45-54,age_group_55-65
0,1,2020-09-01,33,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,2,2022-03-01,53,True,False,False,True,False,False,False,...,False,False,False,False,True,True,False,False,False,False
2,3,2020-10-01,99,False,True,False,False,False,True,False,...,False,False,False,False,True,False,True,False,False,False
3,3,2022-02-01,53,True,False,False,False,False,True,False,...,False,False,False,False,True,False,True,False,False,False
4,4,2022-05-01,85,False,False,True,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18101,10364,2022-09-01,119,False,True,False,False,False,True,False,...,False,False,False,False,True,False,False,False,True,False
18102,10365,2021-05-01,75,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
18103,10365,2021-07-01,43,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
18104,10366,2021-01-01,43,True,False,False,True,False,False,False,...,False,False,False,False,True,True,False,False,False,False


In [20]:
17963+2*8+1*127

18106

In [57]:
#data = pd.read_csv('./drop_dataset.csv')

In [ ]:
#탈퇴한 회원인지 확인하는 열을 추가하고 구독기간을 계산하는 과정 (2023년1월1월기준)

In [21]:
data_initial = data[data['transaction_type_initial']==True] #initial인 행만 추출
tmp = data[data['transaction_type_CHURN']==True]['cust_id'] #탈퇴한 회원의 id 추출

In [22]:
Out_date = data[data['transaction_type_CHURN']==True].reset_index() #탈퇴한 사람들 모아놓은 데이터프레임
in_date = data_initial[data_initial['cust_id'].isin(tmp)].reset_index() #initial이 모여있는 데이터에서 탈퇴한 회원 데이터인 tmp에 id가 존재하는 것들만 뽑아낸 데이터프레임

In [23]:
in_date

,index,cust_id,transaction_date,subscription_price,subscription_type_BASIC,subscription_type_MAX,subscription_type_PRO,customer_country_Denmark,customer_country_Finland,customer_country_Norway,...,referral_type_Organic Search,referral_type_Paid Search,referral_type_TV,referral_type_Unknown,referral_type_facebook,age_group_18-24,age_group_25-34,age_group_35-44,age_group_45-54,age_group_55-65
0,8,8,2021-01-01,43,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
1,10,9,2020-08-01,33,True,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,False
2,23,17,2021-12-01,43,True,False,False,True,False,False,...,False,False,False,True,False,False,False,False,True,False
3,31,22,2022-06-01,53,True,False,False,False,True,False,...,False,False,False,True,False,False,True,False,False,False
4,39,26,2021-02-01,43,True,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,18045,10328,2021-10-01,43,True,False,False,True,False,False,...,False,False,False,False,True,False,False,False,True,False
2048,18062,10340,2020-05-01,65,False,False,True,False,False,False,...,False,False,True,False,False,False,False,True,False,False
2049,18084,10355,2020-05-01,33,True,False,False,False,True,False,...,False,False,True,False,False,False,False,True,False,False
2050,18097,10363,2020-10-01,65,False,False,True,False,True,False,...,False,False,False,False,False,True,False,False,False,False


In [24]:
Out_date

,index,cust_id,transaction_date,subscription_price,subscription_type_BASIC,subscription_type_MAX,subscription_type_PRO,customer_country_Denmark,customer_country_Finland,customer_country_Norway,...,referral_type_Organic Search,referral_type_Paid Search,referral_type_TV,referral_type_Unknown,referral_type_facebook,age_group_18-24,age_group_25-34,age_group_35-44,age_group_45-54,age_group_55-65
0,9,8,2021-04-01,43,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
1,11,9,2021-03-01,43,True,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,False
2,24,17,2022-08-01,53,True,False,False,True,False,False,...,False,False,False,True,False,False,False,False,True,False
3,32,22,2022-11-01,53,True,False,False,False,True,False,...,False,False,False,True,False,False,True,False,False,False
4,41,26,2022-10-01,85,False,False,True,False,False,False,...,False,True,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,18046,10328,2022-04-01,53,True,False,False,True,False,False,...,False,False,False,False,True,False,False,False,True,False
2048,18063,10340,2021-06-01,75,False,False,True,False,False,False,...,False,False,True,False,False,False,False,True,False,False
2049,18085,10355,2020-11-01,33,True,False,False,False,True,False,...,False,False,True,False,False,False,False,True,False,False
2050,18099,10363,2022-02-01,53,True,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False


In [25]:
Out_date['transaction_date'] = pd.to_datetime(Out_date['transaction_date'])
in_date['transaction_date'] = pd.to_datetime(in_date['transaction_date'])

In [26]:
tmp_date = data[data['transaction_type_CHURN']==True].reset_index() #임시 데이터프레임 생성

In [27]:
tmp_date['period'] = Out_date['transaction_date'] - in_date['transaction_date'] #날짜 계산

In [28]:
tmp_date = tmp_date[['cust_id', 'period']] #두 개의 행만 가진 데이터프레임 생성

In [29]:
data2 = pd.merge(data, tmp_date, on='cust_id', how='left')

In [30]:
data2.head(20)

,cust_id,transaction_date,subscription_price,subscription_type_BASIC,subscription_type_MAX,subscription_type_PRO,customer_country_Denmark,customer_country_Finland,customer_country_Norway,customer_country_Sweden,...,referral_type_Paid Search,referral_type_TV,referral_type_Unknown,referral_type_facebook,age_group_18-24,age_group_25-34,age_group_35-44,age_group_45-54,age_group_55-65,period
0,1,2020-09-01,33,True,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,NaT
1,2,2022-03-01,53,True,False,False,True,False,False,False,...,False,False,False,True,True,False,False,False,False,NaT
2,3,2020-10-01,99,False,True,False,False,False,True,False,...,False,False,False,True,False,True,False,False,False,NaT
3,3,2022-02-01,53,True,False,False,False,False,True,False,...,False,False,False,True,False,True,False,False,False,NaT
4,4,2022-05-01,85,False,False,True,False,False,False,True,...,False,False,False,False,True,False,False,False,False,NaT
5,5,2022-08-01,53,True,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,NaT
6,6,2022-06-01,119,False,True,False,False,False,False,True,...,False,False,False,True,False,False,False,False,True,NaT
7,7,2022-02-01,119,False,True,False,False,False,False,True,...,False,False,False,True,False,True,False,False,False,NaT
8,8,2021-01-01,43,True,False,False,False,False,False,True,...,False,False,False,True,False,False,False,True,False,90 days
9,8,2021-04-01,43,True,False,False,False,False,False,True,...,False,False,False,True,False,False,False,True,False,90 days


In [31]:
data2['transaction_date'] = pd.to_datetime(data2['transaction_date'])

In [32]:
Churn = data2[data2['transaction_type_CHURN']==True]['cust_id'].unique() #탈퇴가 기록된 사람들의 id를 뽑아낸 리스트

In [33]:
data2['churn_result'] = data2['cust_id'].apply(lambda x: True if x in Churn else False) #탈퇴했으면 True 유지고객이면 False로 해서 data2에 새로운 'churn_result' 컬럼 추가

In [34]:
c_date = '2023-01-01'#유지중인 고객의 유지 기간을 알기 위해 데이터에 있는 마지막 날짜인 2022년 12월 이후인 2023년 1월 1일로 기준 날짜 설정
c_date = pd.to_datetime(c_date)

In [35]:
data2.loc[data2['period'].isnull(), 'period'] = c_date - data2['transaction_date'] #c_date에서 가입한 날짜를 뺀 것을 data2의 각 줄 period에 저장

In [36]:
# for i in range(len(data2)):
#     if (data2['period'].isnull())[i]==True:
#         data2['period'][i] = c_date - data2['transaction_date'][i]

In [37]:
data2['period'] = data2['period'].astype(str)

In [38]:
data2['period'] = data2['period'].str.replace(' days', '')

In [39]:
data2['period'] = data2['period'].astype(int)

In [40]:
data2['period'] = (data2['period']/30).astype(int)

In [41]:
data2 #최종 데이터 확인

,cust_id,transaction_date,subscription_price,subscription_type_BASIC,subscription_type_MAX,subscription_type_PRO,customer_country_Denmark,customer_country_Finland,customer_country_Norway,customer_country_Sweden,...,referral_type_TV,referral_type_Unknown,referral_type_facebook,age_group_18-24,age_group_25-34,age_group_35-44,age_group_45-54,age_group_55-65,period,churn_result
0,1,2020-09-01,33,True,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,28,False
1,2,2022-03-01,53,True,False,False,True,False,False,False,...,False,False,True,True,False,False,False,False,10,False
2,3,2020-10-01,99,False,True,False,False,False,True,False,...,False,False,True,False,True,False,False,False,27,False
3,3,2022-02-01,53,True,False,False,False,False,True,False,...,False,False,True,False,True,False,False,False,11,False
4,4,2022-05-01,85,False,False,True,False,False,False,True,...,False,False,False,True,False,False,False,False,8,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17958,10364,2022-09-01,119,False,True,False,False,False,True,False,...,False,False,True,False,False,False,True,False,4,False
17959,10365,2021-05-01,75,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,True,20,False
17960,10365,2021-07-01,43,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,True,18,False
17961,10366,2021-01-01,43,True,False,False,True,False,False,False,...,False,False,True,True,False,False,False,False,13,True


In [42]:
data2.corr()['transaction_type_CHURN'].sort_values()

transaction_type_initial       -0.419493
transaction_type_UPGRADE       -0.160995
transaction_type_REDUCTION     -0.145704
period                         -0.122670
customer_country_Norway        -0.105094
customer_country_Sweden        -0.090559
subscription_type_MAX          -0.056335
subscription_price             -0.035873
referral_type_Organic Search   -0.018890
age_group_45-54                -0.016133
customer_gender_Other          -0.012110
customer_gender_Female         -0.010894
cust_id                        -0.006106
referral_type_Display          -0.005193
referral_type_Google Ads        0.000581
age_group_55-65                 0.000839
referral_type_Unknown           0.003015
age_group_35-44                 0.003781
age_group_25-34                 0.004138
subscription_type_PRO           0.004173
referral_type_facebook          0.004796
referral_type_Paid Search       0.005030
referral_type_Bing              0.005981
age_group_18-24                 0.006600
referral_type_TV

In [268]:
#data2.to_csv('period_add_dataset.csv',index=False)